# Neural Collaborative Filtering for Movie Recommendations

Bu notebook, React uygulamasından toplanan kullanıcı puanlamalarını kullanarak Neural Collaborative Filtering (NCF) modeli eğitir.

## Adımlar:
1. Veri yükleme ve ön işleme
2. Model mimarisi oluşturma
3. Model eğitimi
4. Değerlendirme ve görselleştirme
5. Öneri üretme

In [ ]:
# Gerekli kütüphaneleri yükle
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Kütüphaneler yüklendi!")
print(f"TensorFlow version: {tf.__version__}")

## 1. Veri Yükleme ve Keşif

In [ ]:
# React uygulamasından dışa aktarılan JSON dosyasını yükle
# Not: 'ratings.json' dosyasını bu notebook ile aynı klasöre koyun

def load_ratings_data(filepath='ratings.json'):
    """JSON dosyasından puanlama verilerini yükle"""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data['ratings']
    except FileNotFoundError:
        print(f"Dosya bulunamadı: {filepath}")
        print("Örnek veri oluşturuluyor...")
        return generate_sample_data()

def generate_sample_data():
    """Test için örnek veri oluştur"""
    sample_ratings = []
    users = [f"user_{i}" for i in range(1, 26)]  # 25 kullanıcı
    items = list(range(1, 21))  # 20 içerik
    
    for user in users:
        # Her kullanıcı 8-15 rastgele içeriği puanlar
        num_ratings = np.random.randint(8, 16)
        user_items = np.random.choice(items, num_ratings, replace=False)
        
        for item in user_items:
            # Gerçekçi puanlama dağılımı (daha çok 3-5 yıldız)
            rating = np.random.choice([1, 2, 3, 4, 5], p=[0.1, 0.1, 0.2, 0.3, 0.3])
            sample_ratings.append({
                'userId': user,
                'itemId': int(item),
                'rating': int(rating),
                'timestamp': 1234567890,
                'contentType': 'movie'
            })
    
    return sample_ratings

# Veriyi yükle
ratings_data = load_ratings_data()
df = pd.DataFrame(ratings_data)

print(f"Toplam puanlama sayısı: {len(df)}")
print(f"Benzersiz kullanıcı sayısı: {df['userId'].nunique()}")
print(f"Benzersiz içerik sayısı: {df['itemId'].nunique()}")
print(f"Ortalama puanlama: {df['rating'].mean():.2f}")

# İlk 5 satırı göster
df.head()

In [ ]:
# Veri keşfi ve görselleştirme
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Puanlama dağılımı
axes[0, 0].hist(df['rating'], bins=5, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Puanlama Dağılımı')
axes[0, 0].set_xlabel('Puanlama')
axes[0, 0].set_ylabel('Frekans')

# Kullanıcı başına puanlama sayısı
user_counts = df['userId'].value_counts()
axes[0, 1].hist(user_counts, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Kullanıcı Başına Puanlama Sayısı')
axes[0, 1].set_xlabel('Puanlama Sayısı')
axes[0, 1].set_ylabel('Kullanıcı Sayısı')

# İçerik başına puanlama sayısı
item_counts = df['itemId'].value_counts()
axes[1, 0].hist(item_counts, bins=20, alpha=0.7, color='salmon', edgecolor='black')
axes[1, 0].set_title('İçerik Başına Puanlama Sayısı')
axes[1, 0].set_xlabel('Puanlama Sayısı')
axes[1, 0].set_ylabel('İçerik Sayısı')

# Kullanıcı-İçerik matrisi sparsity
total_possible_ratings = df['userId'].nunique() * df['itemId'].nunique()
actual_ratings = len(df)
sparsity = (1 - actual_ratings / total_possible_ratings) * 100

axes[1, 1].bar(['Mevcut Puanlamalar', 'Eksik Puanlamalar'], 
               [actual_ratings, total_possible_ratings - actual_ratings],
               color=['lightblue', 'lightcoral'])
axes[1, 1].set_title(f'Veri Seyrekliği: {sparsity:.1f}%')
axes[1, 1].set_ylabel('Puanlama Sayısı')

plt.tight_layout()
plt.show()

print(f"\nVeri İstatistikleri:")
print(f"Toplam olası puanlama: {total_possible_ratings:,}")
print(f"Mevcut puanlama: {actual_ratings:,}")
print(f"Veri seyrekliği: {sparsity:.1f}%")

## 2. Veri Ön İşleme

In [ ]:
# Kullanıcı ve içerik ID'lerini sayısal değerlere dönüştür
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

df['user_encoded'] = user_encoder.fit_transform(df['userId'])
df['item_encoded'] = item_encoder.fit_transform(df['itemId'])

num_users = df['user_encoded'].nunique()
num_items = df['item_encoded'].nunique()

print(f"Kodlanmış kullanıcı sayısı: {num_users}")
print(f"Kodlanmış içerik sayısı: {num_items}")

# Eğitim ve test verilerini ayır
X = {
    'user_id': df['user_encoded'].values,
    'item_id': df['item_encoded'].values
}
y = df['rating'].values

X_train, X_test, y_train, y_test = train_test_split(
    [X['user_id'], X['item_id']], y, 
    test_size=0.2, 
    random_state=42
)

X_train = {'user_id': X_train[0], 'item_id': X_train[1]}
X_test = {'user_id': X_test[0], 'item_id': X_test[1]}

print(f"\nEğitim verisi boyutu: {len(y_train)}")
print(f"Test verisi boyutu: {len(y_test)}")

# Kodlama örnekleri
print(f"\nKodlama Örnekleri:")
for i in range(3):
    original_user = df.iloc[i]['userId']
    encoded_user = df.iloc[i]['user_encoded']
    original_item = df.iloc[i]['itemId']
    encoded_item = df.iloc[i]['item_encoded']
    print(f"Kullanıcı: {original_user} -> {encoded_user}, İçerik: {original_item} -> {encoded_item}")

## 3. Neural Collaborative Filtering Model Mimarisi

In [ ]:
def build_ncf_model(num_users, num_items, embedding_size=50, hidden_units=[128, 64]):
    """
    Neural Collaborative Filtering modeli oluştur
    
    Args:
        num_users: Benzersiz kullanıcı sayısı
        num_items: Benzersiz içerik sayısı
        embedding_size: Embedding vektör boyutu
        hidden_units: Gizli katman boyutları
    """
    # Giriş katmanları
    user_input = Input(shape=(), name='user_id')
    item_input = Input(shape=(), name='item_id')
    
    # Embedding katmanları
    user_embedding = Embedding(num_users, embedding_size, name='user_embedding')(user_input)
    item_embedding = Embedding(num_items, embedding_size, name='item_embedding')(item_input)
    
    # Embedding'leri düzleştir
    user_vec = Flatten(name='user_flatten')(user_embedding)
    item_vec = Flatten(name='item_flatten')(item_embedding)
    
    # Kullanıcı ve içerik vektörlerini birleştir
    concat = Concatenate(name='concat')([user_vec, item_vec])
    
    # Gizli katmanlar (dropout ile)
    x = concat
    for i, units in enumerate(hidden_units):
        x = Dense(units, activation='relu', name=f'dense_{i+1}')(x)
        x = Dropout(0.2, name=f'dropout_{i+1}')(x)
    
    # Çıkış katmanı (puanlama tahmini)
    output = Dense(1, activation='sigmoid', name='rating_output')(x)
    
    # Çıkışı puanlama aralığına ölçekle (1-5)
    output = tf.keras.layers.Lambda(lambda x: x * 4 + 1, name='scale_output')(output)
    
    # Modeli oluştur
    model = Model(inputs=[user_input, item_input], outputs=output)
    
    # Modeli derle
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Modeli oluştur
embedding_size = 50
hidden_units = [128, 64]

model = build_ncf_model(num_users, num_items, embedding_size, hidden_units)

# Model özetini göster
model.summary()

# Model mimarisini görselleştir
tf.keras.utils.plot_model(model, to_file='ncf_model.png', show_shapes=True, show_layer_names=True)
print("\nModel mimarisi 'ncf_model.png' dosyasına kaydedildi.")

## 4. Model Eğitimi

In [ ]:
# Eğitim parametreleri
epochs = 100
batch_size = 256
validation_split = 0.2

# Callback'ler
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_ncf_model.h5', monitor='val_loss', save_best_only=True)
]

print("Model eğitimi başlıyor...")
print(f"Epochs: {epochs}, Batch Size: {batch_size}")
print(f"Embedding Size: {embedding_size}, Hidden Units: {hidden_units}")

# Modeli eğit
history = model.fit(
    X_train, y_train,
    validation_split=validation_split,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=1
)

print("\nEğitim tamamlandı!")

## 5. Eğitim Sonuçlarını Görselleştir

In [ ]:
# Eğitim geçmişini görselleştir
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss grafiği
ax1.plot(history.history['loss'], label='Eğitim Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Doğrulama Loss', linewidth=2)
ax1.set_title('Model Loss', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MAE grafiği
ax2.plot(history.history['mae'], label='Eğitim MAE', linewidth=2)
ax2.plot(history.history['val_mae'], label='Doğrulama MAE', linewidth=2)
ax2.set_title('Model MAE', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MAE')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# En iyi sonuçları yazdır
best_epoch = np.argmin(history.history['val_loss'])
print(f"\nEn İyi Sonuçlar (Epoch {best_epoch + 1}):")
print(f"Eğitim Loss: {history.history['loss'][best_epoch]:.4f}")
print(f"Doğrulama Loss: {history.history['val_loss'][best_epoch]:.4f}")
print(f"Eğitim MAE: {history.history['mae'][best_epoch]:.4f}")
print(f"Doğrulama MAE: {history.history['val_mae'][best_epoch]:.4f}")

## 6. Model Değerlendirmesi

In [ ]:
# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# Metrikleri hesapla
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Test Verisi Performansı:")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

# Tahmin vs Gerçek değerler grafiği
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
ax1.scatter(y_test, y_pred, alpha=0.6, s=30)
ax1.plot([1, 5], [1, 5], 'r--', linewidth=2)
ax1.set_xlabel('Gerçek Puanlama')
ax1.set_ylabel('Tahmin Edilen Puanlama')
ax1.set_title('Tahmin vs Gerçek Değerler')
ax1.grid(True, alpha=0.3)

# Hata dağılımı
errors = y_test - y_pred.flatten()
ax2.hist(errors, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Hata (Gerçek - Tahmin)')
ax2.set_ylabel('Frekans')
ax2.set_title('Hata Dağılımı')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Puanlama bazında performans
rating_performance = []
for rating in range(1, 6):
    mask = y_test == rating
    if np.sum(mask) > 0:
        rating_mae = mean_absolute_error(y_test[mask], y_pred[mask])
        rating_performance.append({
            'rating': rating,
            'count': np.sum(mask),
            'mae': rating_mae
        })

rating_df = pd.DataFrame(rating_performance)
print(f"\nPuanlama Bazında Performans:")
print(rating_df)

## 7. Öneri Üretme Fonksiyonları

In [ ]:
def predict_rating(model, user_encoder, item_encoder, user_id, item_id):
    """
    Belirli bir kullanıcı-içerik çifti için puanlama tahmini yap
    """
    try:
        user_encoded = user_encoder.transform([user_id])[0]
        item_encoded = item_encoder.transform([item_id])[0]
        
        prediction = model.predict({
            'user_id': np.array([user_encoded]),
            'item_id': np.array([item_encoded])
        }, verbose=0)
        
        return float(prediction[0][0])
    except ValueError:
        # Kullanıcı veya içerik eğitim verisinde yok
        return 3.0  # Varsayılan puanlama

def recommend_items(model, user_encoder, item_encoder, user_id, num_recommendations=10):
    """
    Bir kullanıcı için içerik önerileri üret
    """
    try:
        user_encoded = user_encoder.transform([user_id])[0]
    except ValueError:
        # Yeni kullanıcı - popüler içerikleri döndür
        return list(item_encoder.classes_[:num_recommendations])
    
    # Tüm içerikler için tahmin yap
    all_items = item_encoder.classes_
    item_predictions = []
    
    for item_id in all_items:
        try:
            item_encoded = item_encoder.transform([item_id])[0]
            prediction = model.predict({
                'user_id': np.array([user_encoded]),
                'item_id': np.array([item_encoded])
            }, verbose=0)
            
            item_predictions.append({
                'itemId': item_id,
                'predicted_rating': float(prediction[0][0])
            })
        except:
            continue
    
    # Tahmin edilen puanlamaya göre sırala
    item_predictions.sort(key=lambda x: x['predicted_rating'], reverse=True)
    
    # En iyi önerileri döndür
    return [item['itemId'] for item in item_predictions[:num_recommendations]]

def get_user_recommendations_with_scores(model, user_encoder, item_encoder, user_id, num_recommendations=10):
    """
    Kullanıcı için önerileri puanlarıyla birlikte döndür
    """
    try:
        user_encoded = user_encoder.transform([user_id])[0]
    except ValueError:
        return []
    
    all_items = item_encoder.classes_
    item_predictions = []
    
    for item_id in all_items:
        try:
            item_encoded = item_encoder.transform([item_id])[0]
            prediction = model.predict({
                'user_id': np.array([user_encoded]),
                'item_id': np.array([item_encoded])
            }, verbose=0)
            
            item_predictions.append({
                'itemId': item_id,
                'predicted_rating': float(prediction[0][0])
            })
        except:
            continue
    
    # Tahmin edilen puanlamaya göre sırala
    item_predictions.sort(key=lambda x: x['predicted_rating'], reverse=True)
    
    return item_predictions[:num_recommendations]

print("Öneri fonksiyonları tanımlandı!")

## 8. Örnek Öneriler Üret

In [ ]:
# Rastgele birkaç kullanıcı için öneriler üret
sample_users = list(user_encoder.classes_)[:5]

print("Örnek Kullanıcı Önerileri:")
print("=" * 50)

for user_id in sample_users:
    print(f"\nKullanıcı: {user_id}")
    
    # Kullanıcının mevcut puanlamaları
    user_ratings = df[df['userId'] == user_id][['itemId', 'rating']].sort_values('rating', ascending=False)
    print(f"Mevcut puanlamaları ({len(user_ratings)} adet):")
    for _, row in user_ratings.head(3).iterrows():
        print(f"  İçerik {row['itemId']}: {row['rating']} yıldız")
    
    # Öneriler (puanlarıyla birlikte)
    recommendations = get_user_recommendations_with_scores(model, user_encoder, item_encoder, user_id, 5)
    
    # Kullanıcının daha önce puanlamadığı içerikleri filtrele
    rated_items = set(user_ratings['itemId'].values)
    new_recommendations = [rec for rec in recommendations if rec['itemId'] not in rated_items]
    
    print(f"Yeni öneriler:")
    for i, rec in enumerate(new_recommendations[:3], 1):
        print(f"  {i}. İçerik {rec['itemId']}: {rec['predicted_rating']:.2f} tahmin puanı")
    
    print("-" * 30)

## 9. Model ve Encoderları Kaydet

In [ ]:
import pickle

# Modeli kaydet
model.save('ncf_recommendation_model.h5')
print("Model 'ncf_recommendation_model.h5' dosyasına kaydedildi.")

# Encoderları kaydet
with open('encoders.pkl', 'wb') as f:
    pickle.dump({
        'user_encoder': user_encoder,
        'item_encoder': item_encoder
    }, f)
print("Encoderlar 'encoders.pkl' dosyasına kaydedildi.")

# Model bilgilerini kaydet
model_info = {
    'num_users': num_users,
    'num_items': num_items,
    'embedding_size': embedding_size,
    'hidden_units': hidden_units,
    'test_mse': float(mse),
    'test_mae': float(mae),
    'test_rmse': float(rmse)
}

with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)
print("Model bilgileri 'model_info.json' dosyasına kaydedildi.")

print(f"\nModel Özeti:")
print(f"Kullanıcı sayısı: {num_users}")
print(f"İçerik sayısı: {num_items}")
print(f"Embedding boyutu: {embedding_size}")
print(f"Gizli katmanlar: {hidden_units}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

## 10. Yeni Kullanıcı için Öneri Testi

In [ ]:
# Yeni bir kullanıcı ID'si ile test
test_user_id = "new_user_123"
test_item_id = 1

print(f"Yeni kullanıcı testi: {test_user_id}")

# Puanlama tahmini (yeni kullanıcı için varsayılan değer dönecek)
predicted_rating = predict_rating(model, user_encoder, item_encoder, test_user_id, test_item_id)
print(f"İçerik {test_item_id} için tahmin edilen puanlama: {predicted_rating:.2f}")

# Öneriler (yeni kullanıcı için popüler içerikler dönecek)
recommendations = recommend_items(model, user_encoder, item_encoder, test_user_id, 5)
print(f"Önerilen içerikler: {recommendations}")

# Mevcut bir kullanıcı ile karşılaştırma
existing_user = sample_users[0]
existing_recommendations = recommend_items(model, user_encoder, item_encoder, existing_user, 5)
print(f"\nMevcut kullanıcı ({existing_user}) önerileri: {existing_recommendations}")

print("\n" + "="*60)
print("MODEL EĞİTİMİ TAMAMLANDI!")
print("="*60)
print("Dosyalar:")
print("- ncf_recommendation_model.h5 (Eğitilmiş model)")
print("- encoders.pkl (Kullanıcı/İçerik encoderları)")
print("- model_info.json (Model bilgileri)")
print("- best_ncf_model.h5 (En iyi model checkpoint)")
print("\nBu dosyaları React uygulamanızla entegre edebilirsiniz!")